In [ ]:
import triton
import triton.language as tl
import torch

@triton.jit
def kernel(X, Y, Z, n: tl.constexpr):
    i = tl.program_id(0)
    off = i * 1024 + tl.arange(0, 1024)
    tl.store(Z + off, tl.load(X + off) + tl.load(Y + off))

# compile a kernel instance
import numpy as np
x = torch.randn(1024, device='cuda')
y = torch.randn(1024, device='cuda')
z = torch.empty_like(x)

# force compilation
kernel[(1,)](x, y, z, 1024)
z

In [ ]:
def value(dict_):
    assert len(dict_)==1, 'dict has more than one value' # we're assuming a single env & a single input set
    return list(dict_.values())[0]

In [ ]:
print(value(value(kernel.cache)).asm['ptx'])
# print(value(kernel.device_caches[0][0]).asm["ptx"])

In [ ]:
code = value(value(kernel.cache)).asm['ptx']

def san(x:str):
    return x.replace("%", "%%")
    

print('\n'.join(["\"" + san(x) + "\\n\"" for x in code.split("\n")]))